<a href="https://colab.research.google.com/github/sathsaraRasantha/Question_Answering_for_Sinhala_Language_using_Deep_NN_Architectures/blob/main/corrections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving english_sinhala_combined.csv to english_sinhala_combined.csv


In [4]:
import pandas as pd
data=pd.read_csv("english_sinhala_combined.csv")
data=data.drop(['Unnamed: 0',	'context',	'question',	'text'	],axis=1)

In [10]:
import re
def preprocess_text(document):
        # Remove all the special characters
        document = re.sub(r'\u200d', '', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # define punctuation
        punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''


        preprocessed_text= ""
        for char in document:
          if char not in punctuations:
              preprocessed_text = preprocessed_text+ char
        
        return preprocessed_text

In [13]:
def gather_text_for_vocab(df):
  context=[]
  for row1 in df['context_translated']:
    document1=row1
    text1=preprocess_text(document1)
    context.append(text1)

  
  question=[]
  for row2 in df['question_translated']:
    document2=row2
    text2=preprocess_text(document2)
    question.append(text2)

  answer=[]
  for row3 in df['text_translated']:
    document3=row3
    text3=preprocess_text(document3)
    answer.append(text3)

  lis=context+question
  return lis,context,question,answer

In [19]:
def build_word_vocab(lis):
   word_vocab=[]
   for sentences in lis:
      tokenized=sentences.split()
      word_vocab.append(tokenized)
   
   word2idx = {'__</e>__': 0, '__UNK__': 1} 
   idx2word={}
   for line in lis: 
     words=line.split()
     for word in words:
        if word not in word2idx: 
            word2idx[word] = len(word2idx)

   for item in word2idx:
     idx2word[word2idx[item]]=item


   return word2idx, idx2word, word_vocab

In [20]:
lis,context,question,answer=gather_text_for_vocab(data)

In [23]:
 word2idx, idx2word, word_vocab=build_word_vocab(lis)

In [26]:
len(word2idx)

85814

In [27]:
cleaned_data=pd.DataFrame(list(zip(context,question,answer)), columns =['context_translated', 'question_translated','text_translated'])
cleaned_data

,context_translated,question_translated,text_translated
0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද,උතුරු අත්ලාන්තික්
1,2002 දී ප්රදර්ශනය ආරම්භ වූ දා සිට අයිඩල් ජයග්ර...,ඇමරිකානු අයිඩල් තරඟකරු ක්රිස් ඩොට්රි කුමන ප්රා...,උතුරු කැරොලිනාව
2,3 වන ශතවර්ෂයේ මැද භාගයේ සිට ජර්මානු ගෝත්රිකයන්...,7 වන සියවසේ ඉදිකරන ලද දේ,රෝම බලකොටුවේ බිත්ති ඇතුළත පල්ලියක් ඉදිකරන ලදි
3,රෝම බලකොටුවේ බිත්ති ඇතුළත පල්ලියක් ඉදිකරන ලදි ...,90 දශකයේ දී කැනේ විසින් සාදන ලද බීට්ස් කවුද,දේශීය කලාකරුවන්
4,ටුවාලුආන් භාෂාව සහ ඉංග්රීසි යනු ටුවාලු හි ජාති...,ඉංග්රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුමක්ද,ටුවාලුවාන්
...,...,...,...
19995,දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට සාමාජිකත...,ලිබරල් ප්රජාතන්ත්රවාදයට රාජ්යයක් දක්වන සහයෝගය ...,පිහිටුවීමට සාමාජිකත්වය ප්රකාශ කිරීමට හෝ උද් ca...
19996,පැරණි ඉංග්රීසි භාෂාවෙන් ලතින් භාෂාවෙන් යම් නිශ...,ලතින් හෝඩිය හඳුන්වාදීමට පෙර පැරණි ඉංග්රීසි ලිව...,රූනික් පද්ධතිය
19997,රූනික් පද්ධතිය ඇරිස්ටෝෆනේස්ගේ නාට්ය ග්රීක හාස්...,ප්ලේටෝ සහ ඇරිස්ටෝටල් ලියුවේ කුමන ආකාරයේ සාහිත්...,දාර්ශනික පා
19998,ආරම්භක හා අවසන් දිනයන් දළ වශයෙන් දකුණු අර්ධගෝල...,දකුණු අර්ධගෝලයේ කුමන කාල පරිච්ඡේදයේදී චිලී ප්ර...,ගිම්හානය


In [28]:
answer_start=[]
answer_end=[]

for i in range(0,20000):
  text=str(cleaned_data['context_translated'][i])
  answer=str(cleaned_data['text_translated'][i])
  start=text.find(answer)
  end=start+len(answer)
  answer_start.append(start)
  answer_end.append(end)

In [29]:
cleaned_data['answer_start']=answer_start
cleaned_data['answer_end']=answer_end
cleaned_data

,context_translated,question_translated,text_translated,answer_start,answer_end
0,උතුරු අත්ලාන්තික් සාගරයේ බොස්ටන්හි වෙරළබඩ පිහි...,බොස්ටන් පිහිටා ඇත්තේ කුමන වෙරළ තීරයේද,උතුරු අත්ලාන්තික්,0,17
1,2002 දී ප්රදර්ශනය ආරම්භ වූ දා සිට අයිඩල් ජයග්ර...,ඇමරිකානු අයිඩල් තරඟකරු ක්රිස් ඩොට්රි කුමන ප්රා...,උතුරු කැරොලිනාව,-1,14
2,3 වන ශතවර්ෂයේ මැද භාගයේ සිට ජර්මානු ගෝත්රිකයන්...,7 වන සියවසේ ඉදිකරන ලද දේ,රෝම බලකොටුවේ බිත්ති ඇතුළත පල්ලියක් ඉදිකරන ලදි,-1,44
3,රෝම බලකොටුවේ බිත්ති ඇතුළත පල්ලියක් ඉදිකරන ලදි ...,90 දශකයේ දී කැනේ විසින් සාදන ලද බීට්ස් කවුද,දේශීය කලාකරුවන්,-1,14
4,ටුවාලුආන් භාෂාව සහ ඉංග්රීසි යනු ටුවාලු හි ජාති...,ඉංග්රීසි හැරුණු විට ටුවාලු හි ජාතික භාෂාව කුමක්ද,ටුවාලුවාන්,56,66
...,...,...,...,...,...
19995,දේශපාලන පක්ෂයක අපේක්ෂකයින් පිහිටුවීමට සාමාජිකත...,ලිබරල් ප්රජාතන්ත්රවාදයට රාජ්යයක් දක්වන සහයෝගය ...,පිහිටුවීමට සාමාජිකත්වය ප්රකාශ කිරීමට හෝ උද් ca...,27,101
19996,පැරණි ඉංග්රීසි භාෂාවෙන් ලතින් භාෂාවෙන් යම් නිශ...,ලතින් හෝඩිය හඳුන්වාදීමට පෙර පැරණි ඉංග්රීසි ලිව...,රූනික් පද්ධතිය,-1,13
19997,රූනික් පද්ධතිය ඇරිස්ටෝෆනේස්ගේ නාට්ය ග්රීක හාස්...,ප්ලේටෝ සහ ඇරිස්ටෝටල් ලියුවේ කුමන ආකාරයේ සාහිත්...,දාර්ශනික පා,-1,10
19998,ආරම්භක හා අවසන් දිනයන් දළ වශයෙන් දකුණු අර්ධගෝල...,දකුණු අර්ධගෝලයේ කුමන කාල පරිච්ඡේදයේදී චිලී ප්ර...,ගිම්හානය,304,312


In [30]:
new_df = cleaned_data[cleaned_data.answer_start != -1]
new_df=new_df.reset_index()

len(new_df['context_translated'])

9319

In [31]:
def context_to_ids(text, word2idx):
   
    context_tokens=text.split()
    # for paragraph in stripped:
    #     tokenized=word_tokenize(paragraph)
    #     context_tokens.append(tokenized)
    
    context_ids = [word2idx[word] for word in context_tokens]
    
    assert len(context_ids) == len(context_tokens)
    return context_ids

[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 17,
 38,
 39,
 40,
 32,
 33,
 41,
 42,
 14,
 43,
 44,
 28,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 14,
 56,
 57,
 58,
 59,
 60,
 61,
 4,
 62,
 63,
 52,
 64,
 13,
 14,
 56,
 65,
 66,
 67,
 68,
 69,
 70,
 38,
 13,
 71,
 72,
 15,
 73,
 74,
 75,
 19,
 14,
 76,
 38,
 77,
 15,
 78,
 74,
 14,
 79,
 28,
 14,
 80,
 59,
 67,
 81,
 69,
 82,
 83,
 84,
 85,
 58,
 86,
 87,
 88,
 89,
 90,
 14,
 91,
 92,
 41,
 93,
 94,
 95,
 12,
 96,
 87,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104]

In [34]:
q=0
context_ids=[]
for i in range(0,len(new_df['context_translated'])):
    try:                                                        
        ids = context_to_ids(new_df['context_translated'][i] , word2idx)
        context_ids.append(ids)
        
    except Exception as e:
        context_ids.append(e)
        q=q+1
        continue

In [37]:
len(context_ids)

9319